In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


In [54]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
data_may = pd.read_sql_query('select customer_id, monthname(rental_date) as "May", count(rental_id) from rentals_may group by customer_id', engine)
data_may.head()

,customer_id,May,count(rental_id)
0,130,May,2
1,459,May,1
2,408,May,3
3,333,May,1
4,222,May,5


In [55]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
data_june = pd.read_sql_query('select customer_id, monthname(rental_date) as "June", count(rental_id) from rentals_june group by customer_id', engine)
data_june.head()

,customer_id,June,count(rental_id)
0,416,June,5
1,516,June,6
2,239,June,5
3,285,June,3
4,310,June,6


In [56]:
#data_merged = data_may.merge(data_june, left_on='customer_id', right_on='customer_id')

In [57]:
data_merged = pd.merge(data_may, data_june, left_index=True, right_index=True, how="outer")

In [58]:
data_merged

,customer_id_x,May,count(rental_id)_x,customer_id_y,June,count(rental_id)_y
0,130.0,May,2.0,416,June,5
1,459.0,May,1.0,516,June,6
2,408.0,May,3.0,239,June,5
3,333.0,May,1.0,285,June,3
4,222.0,May,5.0,310,June,6
...,...,...,...,...,...,...
585,NaN,NaN,NaN,412,June,1
586,NaN,NaN,NaN,335,June,1
587,NaN,NaN,NaN,226,June,2
588,NaN,NaN,NaN,22,June,1


In [59]:
data_merged2 = pd.merge(data_may, data_june,on="customer_id", suffixes=("_l", "_r"))
data_merged2

,customer_id,May,count(rental_id)_l,June,count(rental_id)_r
0,130,May,2,June,6
1,459,May,1,June,7
2,408,May,3,June,3
3,333,May,1,June,4
4,222,May,5,June,2
...,...,...,...,...,...
507,431,May,1,June,7
508,191,May,2,June,6
509,351,May,1,June,3
510,10,May,1,June,5


In [60]:
data_merged2 = data_merged2.rename(columns={'count(rental_id)_l': 'may_count', 'count(rental_id)_r' : 'june_count'})
data_merged2

,customer_id,May,may_count,June,june_count
0,130,May,2,June,6
1,459,May,1,June,7
2,408,May,3,June,3
3,333,May,1,June,4
4,222,May,5,June,2
...,...,...,...,...,...
507,431,May,1,June,7
508,191,May,2,June,6
509,351,May,1,June,3
510,10,May,1,June,5


In [61]:
# option 1
df = data_merged2[data_merged2.june_count > data_merged2.may_count]
df

,customer_id,May,may_count,June,june_count
0,130,May,2,June,6
1,459,May,1,June,7
3,333,May,1,June,4
11,261,May,4,June,6
12,334,May,2,June,3
...,...,...,...,...,...
506,234,May,1,June,8
507,431,May,1,June,7
508,191,May,2,June,6
509,351,May,1,June,3


In [64]:
df2 = data_merged2.copy()

In [65]:
# option 2
data_merged2.loc[:, 'more_or_less'] = data_merged2['june_count'] > data_merged2['may_count']
data_merged2

,customer_id,May,may_count,June,june_count,more_or_less
0,130,May,2,June,6,True
1,459,May,1,June,7,True
2,408,May,3,June,3,False
3,333,May,1,June,4,True
4,222,May,5,June,2,False
...,...,...,...,...,...,...
507,431,May,1,June,7,True
508,191,May,2,June,6,True
509,351,May,1,June,3,True
510,10,May,1,June,5,True


In [67]:
def more_or_less(row):
    return row['june_count'] > row['may_count']

df2.loc[:, 'more_or_less'] = df2.apply(more_or_less, axis=1)

In [68]:
df2

,customer_id,May,may_count,June,june_count,more_or_less
0,130,May,2,June,6,True
1,459,May,1,June,7,True
2,408,May,3,June,3,False
3,333,May,1,June,4,True
4,222,May,5,June,2,False
...,...,...,...,...,...,...
507,431,May,1,June,7,True
508,191,May,2,June,6,True
509,351,May,1,June,3,True
510,10,May,1,June,5,True
